# Cyclic Weight Transfer (CWT) with Cyclic Workflow

In this example, we will demonstrate the Cyclic workflow using the Client API with the CIFAR10 dataset. 

## Cyclic Workflow
<a id = "cyclic_workflow"></a>

Cyclic Weight Transfer (CWT) uses the server-controlled `CyclicController` to pass the model weights from one site to the next in a cyclic fashion. 

In the Cyclic workflow, sites train one at a time, while sending the model to the next site. The order of the sites can be specified as fixed, random, or random (without same in a row).  A round is finished once all sites in the defined order have completed training once, and the final result is returned to the server. This differs from Scatter-and-Gather, wherein all sites train simultaneously and aggregate their results together at the end of a round.

## Converting DL training code to FL training code

We will be using the [Client API FL code](../code/fl/train.py) trainer converted from the original [Training a Classifer](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) example.

See [Converting to FL code using Client API](../sag/sag.ipynb#code) for more details.

## Prepare Data

In [ ]:
! python ../data/download.py

## Job Configuration

The client configuration for the trainer with the Client API is standard with the `PTClientAPILauncherExecutor`, `SubprocessLauncher`, and our defined `train.py` that supports the `train` task. This is the same configuration used in the SAG pt workflow.

For the server configuration we use the `CyclicController` for the workflow, and define arguments such as number of rounds, order of relaying sites, and the `train` task name that the clients support. The two required components ids are also mapped to the corresponding `PTFileModelPersistor` and `FullModelShareableGenerator` defined under components.

Let's first copy the required files:

In [ ]:
! cp ../code/fl/train.py train.py
! cp ../code/fl/net.py net.py

Then we can use Job API to easily create a job and run in simulator:

In [ ]:
from net import Net
from nvflare import FedJob
from nvflare.app_common.workflows.cyclic import Cyclic
from nvflare.app_opt.pt.job_config.model import PTModel
from nvflare.job_config.script_runner import FrameworkType, ScriptRunner


if __name__ == "__main__":
    n_clients = 2
    num_rounds = 3
    train_script = "train.py"

    job = FedJob(name="cyclic")

    # Define the controller workflow and send to server
    controller = Cyclic(
        num_clients=n_clients,
        num_rounds=num_rounds,
    )
    job.to(controller, "server")

    # Define the initial global model and send to server
    job.to(PTModel(Net()), "server")

    # Add clients
    for i in range(n_clients):
        runner = ScriptRunner(
            script=train_script,
            script_args="",
        )
        job.to(runner, f"site-{i+1}")

    job.export_job("/tmp/nvflare/jobs")
    job.simulator_run("/tmp/nvflare/jobs/workdir", gpu="0")


Ensure that the `app_script` is set to the Client API FL `train.py` code and the model path for the persistor is set to the `net.Net`.

## Run Job

The previous cell exports the job config and executes the job in NVFlare simulator.

If you want to run in production system, you will need to submit this exported job folder to nvflare system.


As an additional resource, also see the [hello-cyclic](../../../../hello-world/hello-cyclic/README.md) for a Tensorflow Executor implementation using the MNIST dataset.

While this example focused on the server-controlled cyclic workflow, now we will introduce the idea of client-controlled workflows.
The next [cyclic_ccwf](../cyclic_ccwf/cyclic_ccwf.ipynb) example is a client-controlled version of the cyclic workflow.